In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get -y install protobuf-compiler python-pil python-lxml

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-pil
E: Unable to locate package python-lxml


In [ ]:
!pip install openNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 28.3 MB/s eta 0:00:00


In [ ]:
config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Eng.txt.subword.train
        path_tgt: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Eng.txt.subword.test
        path_tgt: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.test
        transforms: [filtertoolong]
# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
#Benchmark value is 10k
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 64
tgt_seq_length: 64

# Tokenization options
src_subword_model: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/source.model
tgt_subword_model: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/target.model
# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.engluganda

# Stop training if it does not imporve after n validations
early_stopping: 10

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 2023

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 6000

# Default: 10000 - Run validation after n steps
valid_steps: 4000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 5000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 1024  # Tokens per batch, change when CUDA out of memory
valid_batch_size: 1024
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.4
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 16
dec_layers: 16
heads: 16
hidden_size: 1024
word_vec_size: 1024
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!onmt_build_vocab -config /content/config.yaml -n_sample -1 -num_threads 2

2023-10-14 16:08:09.994600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 16:08:10.991699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-14 16:08:12.357198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-14 16:08:12.357688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
# Train the NMT model
!onmt_train -config /content/config.yaml

2023-10-14 16:08:21.036270: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 16:08:22.056739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-14 16:08:23.409311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-14 16:08:23.409769: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
!onmt_translate -model /content/models/model.engluganda_step_6000.pt -src /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Eng.txt.subword.dev -output /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/Luganda_TranslatedV11.translated

2023-10-14 17:54:35.274262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 17:54:36.249150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-14 17:54:37.586558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-14 17:54:37.586960: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
!mkdir -p nmt
%cd nmt
!git clone https://github.com/ymoslem/MT-Preparation.git

/content/nmt
Cloning into 'MT-Preparation'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 248 (delta 123), reused 193 (delta 97), pack-reused 0
Receiving objects: 100% (248/248), 64.32 KiB | 12.86 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
# Install the requirements
!pip3 install -r MT-Preparation/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00


In [ ]:
%cd  ../

/content


In [ ]:
!ls

config.yaml  drive  models  nmt  run  sample_data  train.log


In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/target.model  /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/Luganda_TranslatedV11.translated

Done desubwording! Output: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/Luganda_TranslatedV11.translated.desubword


In [ ]:
!ls

config.yaml  drive  models  nmt  run  sample_data  train.log


In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/target.model /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.dev

Done desubwording! Output: /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.dev.desubword


In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
!pip3 install sacrebleu

--2024-04-23 16:23:32--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2024-04-23 16:23:32 (60.4 MB/s) - ‘compute-bleu.py’ saved [957/957]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.4 MB/s eta 0:00:00


In [ ]:
!python3 compute-bleu.py /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.dev.desubword /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/Luganda_TranslatedV11.translated.desubword

Reference 1st sentence: Abnoonyiboobubudamu balina ekifo ekirala kye bayita ewaabwe.
MTed 1st sentence: Abanoonyiboobubudamu balina obuzibu ewaka okuva ewaka.
BLEU:  20.297015608969556


In [ ]:
cat /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/Luganda_TranslatedV11.translated.desubword | sacrebleu /content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.dev.desubword -m bleu chrf ter -b -w 4

[
20.2970,
47.0204,
75.5923
]


# METEOR Score Compmutation

In [ ]:
from nltk.translate.meteor_score import meteor_score, single_meteor_score
import nltk

In [ ]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('punkt')

# Function to read the content of a text file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Tokenize the text
def tokenize(text):
    return nltk.word_tokenize(text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Function to read the content of a text file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Paths to your text files
file1_path = '/content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/English_Luganda_Lug.txt.subword.dev.desubword'
file2_path = '/content/drive/MyDrive/LanguageTranslationData_New/EngLuganda_Kimera_Benchmark/Luganda_TranslatedV11.translated.desubword'

# Read the contents of the files
reference = read_file(file1_path)
hypothesis = read_file(file2_path)

# Tokenize the texts
reference_tokens = tokenize(reference)
hypothesis_tokens = tokenize(hypothesis)

# Calculate the METEOR score
score = meteor_score([reference_tokens], hypothesis_tokens)

print(f'The METEOR score between the two files is: {score}')

The METEOR score between the two files is: 0.39789450317024894
